In [3]:
## Runs my implementation of Word2Vec
## The input is a text file plus some parameters 
## The pre-processing is performed by functions in TexttoDict

import numpy as np
import theano
import theano.tensor as T
from theano import shared
import nltk
import string
from collections import defaultdict
from TexttoDict import *

rng = np.random

In [56]:
#---------------------------------------
# C is the size of the bag of words
# N is the size of hidden layer
# V is length of dictcut

class NNet(object):
    def __init__(self,V,N):

        # inititalizing the weight matrices    
        self.W1 = theano.shared(rng.randn(V,N), name="W1")   # a V x N matrix
        self.W2 = theano.shared(rng.randn(V,N), name="W2")   # a V x N matrix

# (x,y) are shared variables for (input, output)
# x is the input bag of words
# y is the target word, the middle word in the bag
        x = T.ivector("x") # a vector
        y = T.ivector("y") # a vector

# This evaluates the hidden and output layers
# This is a linear operation, no logistic function unlike a typical neural net
# To be more precise we are evaluating these at the level of the hidden layer
        vhidden = self.W1[x] # a C x N matrix
        vout = self.W2[y] # a 1 x N matrix

        vhiddenvec= T.mean(vhidden,axis=0)  # an N-vector
        voutvec=vout.reshape((N,)) # an N-vector
    
# Entropy loss function (a scalar)
        Entropy = -T.dot(vhiddenvec,voutvec) + T.log(T.exp(T.dot(vhidden,self.W2.T)).sum())

# Compute the gradient of the cost wrt the weights
# This is the computational power of Theano
        gradW1, gradW2 = T.grad(Entropy, [self.W1, self.W2])

# The train function updates the weights and computes the entropy
        train = theano.function(
              inputs=[x,y],
              outputs=Entropy,
              updates=((self.W1, self.W1 - 0.1 * gradW1),(self.W2, self.W2 - 0.1 * gradW2)))
    
        self.train=train

#---------------------------------------
    def NNettrain(self,inarr, outarr):
        for i in range(len(inarr)):
            ent=self.train(inarr[i], outarr[i])
    
        return ent
    
    def returnweights(self):
        return self.W1


This provides the bags of words from the textfile
using functions from TexttoDict.py

In [67]:
W2Vdata=w2vdatafromtext("delorme.txt", contsize=7, trainlines=10000,freqcutoff=1)
V=len(W2Vdata[0])
N=10
MyNet=NNet(V,N)

In [68]:
error=MyNet.NNettrain(W2Vdata[1], W2Vdata[2])
weights= MyNet.returnweights()

In [69]:
print error
print weights.get_value()

7.06137366028
[[ 0.5299108  -0.50288172  0.17919674 ..., -0.26912587 -0.83869701
   1.15402314]
 [-0.07612862  0.36523929  0.56889645 ..., -0.1217904   1.00406584
   0.94851183]
 [-0.05125551 -0.7749254  -1.12739845 ..., -1.22261318  0.41440998
  -0.14258977]
 ..., 
 [ 0.25960022 -0.30468155  0.31813367 ..., -0.43372954 -1.30358448
  -1.11348332]
 [-0.26375517  1.00572165 -0.22273296 ...,  0.2810096  -0.37474531
   0.79309478]
 [ 0.53504905  0.67036436  0.66853088 ...,  0.85988386  0.16894902
   0.55646764]]
